In [ ]:
#Downloaded from doug.specht.co.uk/twitter-research

In [20]:
import sys
import json
import re
import numpy as np
from datetime import datetime
import pandas as pd  

tweetfile = 'beijing2022.csv'

tweets = pd.read_csv(tweetfile)

Please verify that these are the headers to be used: ['from_user', 'created_at']. 'retweets' are taken from the 'text' header where 'RT @' is mentioned.

In [21]:
# 1. Export edges from Retweets

tweets['retweets'] = tweets['text'].str.extract('RT @([a-zA-Z0-9]\w{0,}):', expand=True)

edges = tweets[['from_user', 'retweets','user_followers_count']]
edges.columns = ['Source', 'Target', 'Strength']

In [22]:
strengthLevel = 3  # Network connection strength level: the number of times in total each of the tweeters responded to or mentioned the other.
                   # If you have 1 as the level, then all tweeters who mentioned or replied to another at least once will be displayed. But if you have 5, only those who have mentioned or responded to a particular tweeter at least 5 times will be displayed, which means that only the strongest bonds are shown.

edges2 = edges.groupby(['Source','Target'])['Strength'].count()
edges2 = edges2.reset_index()
edges2 = edges2[edges2['Strength'] >= strengthLevel]

Please verify that these are the headers to be used: ['from_user','user_followers_count','user_friends_count']

In [23]:
# Export nodes from the edges and add node attributes for both Sources and Targets.

users = tweets[['from_user','user_followers_count','user_friends_count']]
users = users.sort_values(['from_user','user_followers_count'], ascending=[True, False])
users = users.drop_duplicates(['from_user'], keep='first') 

ids = edges2['Source'].append(edges2['Target']).to_frame()
ids['Label'] = ids
ids.columns = ['from_user', 'Label']
ids = ids.drop_duplicates(['from_user'], keep='first') 
nodes = pd.merge(ids, users, on='from_user', how='left')

In [24]:
print(nodes.shape)
print(edges2.shape)

(3, 4)
(2, 3)


In [25]:
# change column names for Kumu import (Run this when using Kumu)
edges2.columns = ['Source','Target','Strength']
nodes.columns = ['Id', 'Label', 'user_followers_count', 'user_friends_count']

In [26]:
# print edges to check 
edges2

,Source,Target,Strength
108,Tobi97Tobi,Beijing2022,76
137,c23cg,Beijing2022,3


In [27]:
# print nodes to check
nodes

,Id,Label,user_followers_count,user_friends_count
0,Tobi97Tobi,Tobi97Tobi,5.0,8.0
1,c23cg,c23cg,10.0,56.0
2,Beijing2022,Beijing2022,99579.0,1053.0


In [28]:
# Export nodes and edges to csv files
nodes.to_csv('beijing2022.csv', encoding='utf-8', index=False)
edges.to_csv('beijing2022.csv', encoding='utf-8', index=False)